In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.holtwinters import ExponentialSmoothing


In [ ]:
## Load the dataset
# Replace 'superstore_sales.csv' with the path to your dataset file
df = pd.read_csv('superstore_sales.csv')



In [ ]:
# Data Cleaning and Preparation

# Display the initial dataset
print("Initial Dataset:")
print(df.head())



In [ ]:
# Remove Duplicates
duplicates = df.duplicated().sum()
print(f"\nNumber of duplicate rows: {duplicates}")
df = df.drop_duplicates()
print(f"Dataset shape after removing duplicates: {df.shape}")



In [ ]:
# Correct Errors in Data Entries
negative_sales = df[df['Sales'] < 0]
negative_quantity = df[df['Quantity'] < 0]
print(f"\nNumber of transactions with negative sales: {len(negative_sales)}")
print(f"Number of transactions with negative quantity: {len(negative_quantity)}")
df = df[df['Sales'] >= 0]  # Remove negative sales
df = df[df['Quantity'] >= 0]  # Remove negative quantity



In [ ]:
# Handle Missing Values
missing_values = df.isnull().sum()
print("\nMissing values in each column:")
print(missing_values[missing_values > 0])
df['Sales'].fillna(df['Sales'].mean(), inplace=True)
df['Quantity'].fillna(df['Quantity'].mean(), inplace=True)
df['Profit'].fillna(df['Profit'].mean(), inplace=True)
df['Segment'].fillna(df['Segment'].mode()[0], inplace=True)
df['Ship Mode'].fillna(df['Ship Mode'].mode()[0], inplace=True)
print(f"\nDataset shape after handling missing values: {df.shape}")



In [ ]:
# Create Additional Columns for Time-Based Analysis
df['Order Date'] = pd.to_datetime(df['Order Date'])
df['Order Year'] = df['Order Date'].dt.year
df['Order Quarter'] = df['Order Date'].dt.quarter



In [ ]:
# Data Modeling

# Create Products Table
products = df[['Product ID', 'Product Name', 'Category', 'Sub-Category']].drop_duplicates()
products.reset_index(drop=True, inplace=True)



In [ ]:
# Create Customers Table
customers = df[['Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State']].drop_duplicates()
customers.reset_index(drop=True, inplace=True)



In [ ]:
# Create Orders Table
orders = df[['Order ID', 'Order Date', 'Ship Date', 'Customer ID', 'Product ID', 'Sales', 'Quantity', 'Profit']].drop_duplicates()
orders.reset_index(drop=True, inplace=True)



In [ ]:
# Create Shipping Table
shipping = df[['Ship Mode']].drop_duplicates()
shipping.reset_index(drop=True, inplace=True)
shipping['Shipping Cost'] = [10.00 if mode == "Standard Class" else 20.00 for mode in shipping['Ship Mode']]



In [ ]:
# Create Regions Table
regions = df[['Region', 'State']].drop_duplicates()
regions.reset_index(drop=True, inplace=True)



In [ ]:
# Metrics Selection

# Calculate Total Sales Revenue
total_sales_revenue = df['Sales'].sum()
print(f"\nTotal Sales Revenue: ${total_sales_revenue:.2f}")



In [ ]:
# Calculate Profit Margin
df['Profit Margin'] = (df['Profit'] / df['Sales']) * 100
average_profit_margin = df['Profit Margin'].mean()
print(f"Average Profit Margin: {average_profit_margin:.2f}%")



In [ ]:
# Sales by Category
sales_by_category = df.groupby('Category')['Sales'].sum().reset_index()
sales_by_category.sort_values(by='Sales', ascending=False, inplace=True)



In [ ]:
# Sales by Region
sales_by_region = df.groupby('Region')['Sales'].sum().reset_index()
sales_by_region.sort_values(by='Sales', ascending=False, inplace=True)



In [ ]:
# Customer Segmentation
sales_by_segment = df.groupby('Segment')['Sales'].sum().reset_index()
sales_by_segment.sort_values(by='Sales', ascending=False, inplace=True)



In [ ]:
# Visualization of Metrics

# Plotting Sales by Category
plt.figure(figsize=(10, 6))
sns.barplot(x='Sales', y='Category', data=sales_by_category, palette='viridis')
plt.title('Sales by Category')
plt.xlabel('Total Sales')
plt.ylabel('Category')
plt.show()



In [ ]:
# Plotting Sales by Region
plt.figure(figsize=(10, 6))
sns.barplot(x='Sales', y='Region', data=sales_by_region, palette='lightgreen')
plt.title('Sales by Region')
plt.xlabel('Total Sales')
plt.ylabel('Region')
plt.show()



In [ ]:
# Plotting Sales by Customer Segment
plt.figure(figsize=(10, 6))
sns.barplot(x='Sales', y='Segment', data=sales_by_segment, palette='Blues')
plt.title('Sales by Customer Segment')
plt.xlabel('Total Sales')
plt.ylabel('Customer Segment')
plt.show()



In [ ]:
# Analysis and Interpretation

## Sales Trends: Identify Seasonal Trends or Peak Sales Periods
monthly_sales = df.resample('M')['Sales'].sum()
plt.figure(figsize=(12, 6))
plt.plot(monthly_sales, marker='o')
plt.title('Monthly Sales Trend')
plt.xlabel('Month')
plt.ylabel('Total Sales')
plt.xticks(rotation=45)
plt.grid()
plt.show()



In [ ]:
## Identify peak sales periods
peak_month = monthly_sales.idxmax()
peak_sales = monthly_sales.max()
print(f"Peak Sales Month: {peak_month.strftime('%B %Y')} with Sales: ${peak_sales:.2f}")



In [ ]:
## Product Performance: Top Sellers and Underperforming Products
top_products = df.groupby('Product Name')['Sales'].sum().nlargest(10).reset_index()
underperforming_products = df.groupby('Product Name')['Sales'].sum().nsmallest(10).reset_index()

plt.figure(figsize=(12, 6))
sns.barplot(x='Sales', y='Product Name', data=top_products, palette='viridis')
plt.title('Top 10 Selling Products')
plt.xlabel('Total Sales')
plt.ylabel('Product Name')
plt.show()



In [ ]:
plt.figure(figsize=(12, 6))
sns.barplot(x='Sales', y='Product Name', data=underperforming_products, palette='rocket')
plt.title('Underperforming Products')
plt.xlabel('Total Sales')
plt.ylabel('Product Name')
plt.show()



In [ ]:
## Customer Behavior: Purchasing Patterns Among Different Customer Segments
sales_by_segment = df.groupby('Segment')['Sales'].sum().reset_index()

plt.figure(figsize=(8, 5))
sns.barplot(x='Segment', y='Sales', data=sales_by_segment, palette='Blues')
plt.title('Sales by Customer Segment')
plt.xlabel('Customer Segment')
plt.ylabel('Total Sales')
plt.show()



In [ ]:
# Recommendations

print("\nRecommendations:")
print("1. Tailor marketing efforts towards high-performing product categories.")
print("2. Focus on Consumer segment as it shows significant sales.")
print("3. Optimize stock levels based on sales forecasts to minimize stockouts.")
print("4. Adjust discount strategies based on their impact on profit margins.")



In [ ]:
# Sales Forecasting

## Implement Time Series Forecasting using Exponential Smoothing
model = ExponentialSmoothing(monthly_sales, trend='add', seasonal='add', seasonal_periods=12)
fit_model = model.fit()



In [ ]:
# Forecast future sales for the next 12 months
forecast = fit_model.forecast(steps=12)
forecast_index = pd.date_range(start=monthly_sales.index[-1] + pd.DateOffset(months=1), periods=12, freq='M')

# Plotting the forecasted values along with historical data
plt.figure(figsize=(12, 6))
plt.plot(monthly_sales, label='Historical Sales', marker='o')
plt.plot(forecast_index, forecast, label='Forecasted Sales', marker='o', color='orange')
plt.title('Sales Forecast for Next 12 Months')
plt.xlabel('Month')
plt.ylabel('Total Sales')
plt.legend()
plt.grid()
plt.show()



In [ ]:
# Continuous Improvement

print("\nContinuous Improvement Process:")
print("1. Regularly update datasets and refine analysis based on new data.")
print("2. Utilize feedback from stakeholders to adjust strategies as needed.")